In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from pandas.tseries.offsets import BusinessDay

In [2]:
sp = yf.download('^SPX')

sp.index = pd.to_datetime(sp.index)

# Filter the DataFrame
start_date = '2000-01-01'
end_date = '2023-08-31'
filtered_sp = sp.loc[start_date:end_date]
# Remove the 'Ticker' level from the MultiIndex
filtered_sp.columns = filtered_sp.columns.droplevel(1)
filtered_sp = pd.DataFrame(filtered_sp['Close'], index=filtered_sp.index)
filtered_sp.index = pd.to_datetime(filtered_sp.index).tz_localize(None)

[*********************100%***********************]  1 of 1 completed


In [3]:
data = pd.read_csv('filtered_data2.csv')
data.set_index('Date', inplace=True)

In [4]:
data

,exdate,cp_flag,strike_price,best_bid,best_offer,volume,impl_volatility,delta,gamma,optionid,contract_size,D to Expiration,Close,Moneyness
Date,,,,,,,,,,,,,,
2000-01-03,2000-01-22,P,1465.0,30.000,32.000,68,0.218037,-0.521459,0.005654,10526329,100,19,1455.219971,1.006721
2000-01-03,2000-01-22,P,1440.0,19.875,21.125,691,0.227943,-0.385638,0.005192,10563021,100,19,1455.219971,0.989541
2000-01-03,2000-01-22,C,1460.0,23.375,25.375,386,0.191068,0.505087,0.006461,10949021,100,19,1455.219971,1.003285
2000-01-03,2000-01-22,C,1450.0,29.125,32.000,1312,0.198929,0.567313,0.006117,11070856,100,19,1455.219971,0.996413
2000-01-03,2000-01-22,C,1465.0,20.625,22.625,0,0.187986,0.472246,0.006551,11274505,100,19,1455.219971,1.006721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-31,2023-09-29,P,4435.0,28.800,29.100,21,0.128259,-0.281094,0.002070,154635934,100,29,4507.660156,0.983881
2023-08-31,2023-10-02,C,4515.0,65.400,65.900,0,0.111276,0.540238,0.002672,156631215,100,32,4507.660156,1.001628
2023-08-31,2023-10-02,P,4515.0,53.500,53.900,12,0.112415,-0.459972,0.002645,156631264,100,32,4507.660156,1.001628


In [5]:
data['Midprice'] = (data['best_bid'] + data['best_offer']) / 2

def calendar_to_business_days(start_date, end_date, calendar_days):

    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    # Ensure start_date is in string format
    start_date_str = start_date.strftime('%Y-%m-%d')
    # Calculate the end date
    end_date_str = end_date.strftime('%Y-%m-%d')
    # Calculate business days
    business_days = np.busday_count(start_date_str, end_date_str)
    return business_days

# Apply the function to the 'D to Expiration' column
data['Business_Days_to_Expiration'] = data.apply(lambda row: calendar_to_business_days(row.name, row['exdate'], row['D to Expiration']), axis=1)

In [6]:
data_1m = data[data['Business_Days_to_Expiration'] >= 22]
data_1m.index = pd.to_datetime(data_1m.index)

In [7]:
data_1m

,exdate,cp_flag,strike_price,best_bid,best_offer,volume,impl_volatility,delta,gamma,optionid,contract_size,D to Expiration,Close,Moneyness,Midprice,Business_Days_to_Expiration
Date,,,,,,,,,,,,,,,,
2000-01-11,2000-02-19,C,1465.0,27.625,29.625,10,0.197147,0.437028,0.004305,10136376,100,39,1438.560059,1.018379,28.625,29
2000-01-11,2000-02-19,C,1400.0,68.625,70.625,0,0.230368,0.686109,0.003317,10150117,100,39,1438.560059,0.973195,69.625,29
2000-01-11,2000-02-19,C,1425.0,50.625,52.625,27,0.215107,0.602236,0.003864,10156700,100,39,1438.560059,0.990574,51.625,29
2000-01-11,2000-02-19,C,1450.0,35.250,37.250,3917,0.202589,0.501965,0.004242,10251188,100,39,1438.560059,1.007952,36.250,29
2000-01-11,2000-02-19,P,1465.0,43.375,45.375,38,0.187451,-0.567462,0.004519,10972804,100,39,1438.560059,1.018379,44.375,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-30,2023-10-06,P,4510.0,57.100,57.600,28,0.122389,-0.431375,0.002233,156187017,100,37,4514.870117,0.998921,57.350,27
2023-08-31,2023-10-02,C,4515.0,65.400,65.900,0,0.111276,0.540238,0.002672,156631215,100,32,4507.660156,1.001628,65.650,22
2023-08-31,2023-10-02,P,4515.0,53.500,53.900,12,0.112415,-0.459972,0.002645,156631264,100,32,4507.660156,1.001628,53.700,22


In [8]:
# Group the data by date, strike price, and expiration date
grouped = data_1m.groupby(['Date', 'strike_price', 'Business_Days_to_Expiration'])

# Create straddles
straddles = []

for (date, strike_price, day_to_exp), group in grouped:
    if len(group) == 2:
        call = group[group['cp_flag'] == 'C']
        put = group[group['cp_flag'] == 'P']
        if not call.empty and not put.empty:
            straddle = {
                'Date': pd.to_datetime(date),
                'Strike_Price': strike_price,
                'D to Expiration': int(day_to_exp),
                'Call_Midprice': call['Midprice'].values[0],
                'Put_Midprice': put['Midprice'].values[0],
                'Total_Cost': call['Midprice'].values[0] + put['Midprice'].values[0],
                'Underlying Price': call['Close'].values[0],
                'Call_Delta': call['delta'].values[0],
                'Put_Delta': put['delta'].values[0],
                'Call_Optionid': call['optionid'].values[0],
                'Put_Optionid': put['optionid'].values[0]
            }
            straddles.append(straddle)

# Convert the list of straddles to a DataFrame
straddles_df = pd.DataFrame(straddles)
straddles_df['Date'] = pd.to_datetime(straddles_df['Date'])
straddles_df.set_index('Date', inplace=True)

In [9]:
for dates in straddles_df.index:
    if dates.day > 10:
        straddles_df.drop(dates, inplace=True, errors='ignore')
straddles_df

,Strike_Price,D to Expiration,Call_Midprice,Put_Midprice,Total_Cost,Underlying Price,Call_Delta,Put_Delta,Call_Optionid,Put_Optionid
Date,,,,,,,,,,
2000-02-08,1350.0,29,107.000,10.500,117.500,1441.719971,0.835956,-0.170638,11270247,11630621
2000-02-08,1365.0,29,94.125,12.625,106.750,1441.719971,0.803916,-0.202492,10026760,10062046
2000-02-08,1390.0,29,73.875,17.125,91.000,1441.719971,0.736453,-0.267485,10910871,10184183
2000-02-08,1395.0,29,70.250,18.125,88.375,1441.719971,0.719612,-0.282109,11037616,10384710
2000-02-08,1400.0,29,66.375,19.125,85.500,1441.719971,0.703825,-0.297189,10126667,11305405
...,...,...,...,...,...,...,...,...,...,...
2023-08-09,4510.0,22,53.350,81.050,134.400,4467.709961,0.445052,-0.552319,155952151,155952233
2023-08-10,4370.0,26,155.950,40.450,196.400,4468.830078,0.720425,-0.285514,153998536,153998764
2023-08-10,4390.0,26,139.600,44.150,183.750,4468.830078,0.691232,-0.312958,153649185,153649192


In [10]:
straddles_reset = straddles_df.reset_index()
straddles_reset['YearMonth'] = straddles_reset['Date'].dt.to_period('M')
straddles_reset.sort_values(['YearMonth', 'Date'], inplace=True)
first_straddles_daily = straddles_reset.drop_duplicates(subset='YearMonth', keep='first')
first_straddles_daily.set_index('Date', inplace=True)
first_straddles_daily.drop(columns=['YearMonth'], inplace=True)

first_straddles_daily

C:\Users\benj0\AppData\Local\Temp\ipykernel_80960\1657466932.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_straddles_daily.drop(columns=['YearMonth'], inplace=True)


,Strike_Price,D to Expiration,Call_Midprice,Put_Midprice,Total_Cost,Underlying Price,Call_Delta,Put_Delta,Call_Optionid,Put_Optionid
Date,,,,,,,,,,
2000-02-08,1350.0,29,107.000,10.500,117.500,1441.719971,0.835956,-0.170638,11270247,11630621
2000-05-09,1375.0,29,73.875,30.125,104.000,1412.140015,0.657498,-0.342493,10901690,10899368
2000-08-08,1460.0,29,52.500,20.625,73.125,1482.800049,0.658420,-0.342908,10556436,11233358
2000-10-10,1325.0,29,79.500,20.500,100.000,1387.020020,0.730977,-0.275124,10818409,10740154
2000-11-07,1310.0,29,141.250,8.750,150.000,1431.869995,0.857221,-0.131243,11905748,11905749
...,...,...,...,...,...,...,...,...,...,...
2023-04-03,4055.0,24,136.050,51.900,187.950,4124.509766,0.659954,-0.340454,153014313,153014361
2023-05-01,4060.0,22,150.100,29.150,179.250,4167.870117,0.753633,-0.250283,151174797,151174923
2023-06-01,4285.0,22,33.050,85.450,118.500,4221.020020,0.364524,-0.625927,154801516,154801527


In [ ]:
# Implement Delta Hedging
def delta_hedging(straddles_df, rf_rate, transaction_cost=0):
    results = []
    for index, row in straddles_df.iterrows():
        strike_price = row['Strike_Price']
        # Get the id to retrieve the right delta
        call_id = row['Call_Optionid']
        put_id = row['Put_Optionid']
        # Select right delta for our options
        mask_call = (data['optionid'] == call_id) & (data['cp_flag'] == 'C')
        mask_put = (data['optionid'] == put_id) & (data['cp_flag'] == 'P')
        # Basic required data for time 0
        total_cost = row['Total_Cost']
        underlying_price = row['Underlying Price']
        call_delta = row['Call_Delta']
        put_delta = row['Put_Delta']
        delta0 = -(call_delta + put_delta)
        initial_value = total_cost
        initial_capital = initial_value
        initial_cash = initial_capital - (delta0 * underlying_price)
        positions = []
        values = []
        
        for i in range(int(row['D to Expiration'])):
            # Calculate the target date by adding 'i' business days
            target_date = index + BusinessDay(n=i)
            
            # Ensure the target_date is within the index of filtered_sp
            if target_date not in filtered_sp.index:
                print(f"Warning: Target date {target_date} not found in filtered_sp index.")
                underlying_price_i = underlying_price.loc[target_date - BusinessDay(1)] ### REPRENDRE ICIC ET METTRE DATA['CLOSE']
            else:
                # Retrieve the underlying price for the target date
                underlying_price_i = filtered_sp.loc[target_date]

            # Retrieve current and previous deltas if available
            if i == 0:
                previous_delta = delta0
            else:
                previous_target_date = index + BusinessDay(n=i-1)
                if previous_target_date not in filtered_sp.index:
                    print(f"Warning: Previous target date {previous_target_date} not found in filtered_sp index.")
                    previous_delta = delta0  
                else:
                    previous_call_delta = data.loc[mask_call & (data.index == previous_target_date), 'delta']
                    previous_put_delta = data.loc[mask_put & (data.index == previous_target_date), 'delta']
                    previous_delta = -(previous_call_delta + previous_put_delta)
                    
            # Calculate current delta
            current_call_delta = data.loc[mask_call & (data.index == target_date), 'delta']
            current_put_delta = data.loc[mask_put & (data.index == target_date), 'delta']
            current_delta = -(current_call_delta + current_put_delta)
            
            # Calculate hedged position
            hedged_position = current_delta * underlying_price_i
            
            # Update capital
            delta_change = previous_delta - current_delta
            capital = initial_cash * np.exp(rf_rate/252) + (delta_change * underlying_price_i) - (transaction_cost * abs(delta_change) * underlying_price_i)
            
            # Calculate portfolio value
            value = capital + hedged_position
            positions.append(hedged_position)
            values.append(value)
        
        # Calculate final value (this logic might need adjustment based on your strategy)
        final_value = values[-1] if values else initial_cash
        profit_loss = final_value - initial_value
        
        results.append({
            'Date': index,
            'Initial Value': initial_value,
            'Final Value': final_value,
            'Profit/Loss': profit_loss
        })
        
    return pd.DataFrame(results)

In [12]:
# Delta Hedging without transaction costs
results_no_cost = delta_hedging(straddles_df, 0.001, transaction_cost=0)
results_no_cost

AttributeError: 'numpy.float64' object has no attribute 'loc'